# Bike share system EDA: Trip Time Series Data 2017-Present

In [1]:
# intended to be submitted via pyspark but produce results on the command line.

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, LongType
from pyspark.sql.functions import expr, col, column, lit, to_date, coalesce, to_timestamp

import plotly.express as px

In [2]:
import os
memory = '20g'
pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

In [3]:
def to_timestamp_(col, formats=("MM/dd/yyyy hh:mm:ss", "yyyy-MM-dd hh:mm:ss")):
    # Spark 2.2 or later syntax, for < 2.2 use unix_timestamp and cast
    return coalesce(*[to_timestamp(col, f) for f in formats])

In [4]:
spark = SparkSession.builder.appName("Sample bike data").getOrCreate()
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

df = spark.read.load("alldata.parquet")
df.createOrReplaceTempView("bikedata")

In [5]:
# # we only care about data before 2017
# bike_data_sql = """
# select TO_DATE(starttime) trip_date, city, (unix_timestamp(stoptime) - unix_timestamp(starttime))/60 as trip_duration_minutes 
# from bikedata
# WHERE starttime >= cast('2017-01-01 00:00:00' AS TIMESTAMP)
# """

# df = spark.sql(bike_data_sql)
# df.createOrReplaceTempView("bikedata")

In [6]:
bike_data_sql = """
select TO_DATE(starttime) trip_date, city, (unix_timestamp(stoptime) - unix_timestamp(starttime))/60 as trip_duration_minutes 
from bikedata
WHERE starttime >= cast('2017-01-01 00:00:00' AS TIMESTAMP)
"""

df = spark.sql(bike_data_sql)
df.createOrReplaceTempView("bikedata")

# filter out trips longer than 24 hours
bike_data_sql = """
select *
from bikedata
WHERE trip_duration_minutes < 1440
"""

df = spark.sql(bike_data_sql)
df.createOrReplaceTempView("bikedata")

df.show(20, False)

+----------+------+---------------------+
|trip_date |city  |trip_duration_minutes|
+----------+------+---------------------+
|2021-09-01|boston|10.85                |
|2021-09-01|boston|16.033333333333335   |
|2021-09-01|boston|13.35                |
|2021-09-01|boston|17.166666666666668   |
|2021-09-01|boston|18.55                |
|2021-09-01|boston|8.15                 |
|2021-09-01|boston|38.5                 |
|2021-09-01|boston|4.8                  |
|2021-09-01|boston|17.8                 |
|2021-09-01|boston|6.283333333333333    |
|2021-09-01|boston|5.416666666666667    |
|2021-09-01|boston|68.5                 |
|2021-09-01|boston|8.016666666666667    |
|2021-09-01|boston|8.15                 |
|2021-09-01|boston|9.866666666666667    |
|2021-09-01|boston|6.733333333333333    |
|2021-09-01|boston|2.0166666666666666   |
|2021-09-01|boston|6.066666666666666    |
|2021-09-01|boston|2.8333333333333335   |
|2021-09-01|boston|14.716666666666667   |
+----------+------+---------------

In [118]:
trip_data_group_by_date_sql = """
SELECT 
    city,
    trip_date,
    percentile(trip_duration_minutes, 0.5) median_trip_duration_minutes,
    count(*) as trip_count
FROM bikedata
GROUP BY city, trip_date
ORDER BY city, trip_date"""


trip_data_group_by_date_df = spark.sql(trip_data_group_by_date_sql)

trip_data_group_by_date_df.show(20, False)

+------+----------+----------------------------+----------+
|city  |trip_date |median_trip_duration_minutes|trip_count|
+------+----------+----------------------------+----------+
|boston|2017-01-01|9.216666666666667           |481       |
|boston|2017-01-02|8.7                         |801       |
|boston|2017-01-03|8.633333333333333           |651       |
|boston|2017-01-04|9.016666666666667           |1534      |
|boston|2017-01-05|8.7                         |1330      |
|boston|2017-01-06|8.083333333333334           |835       |
|boston|2017-01-07|7.175000000000001           |106       |
|boston|2017-01-08|7.866666666666666           |111       |
|boston|2017-01-09|7.091666666666667           |392       |
|boston|2017-01-10|7.233333333333333           |469       |
|boston|2017-01-11|7.566666666666666           |773       |
|boston|2017-01-12|6.933333333333334           |763       |
|boston|2017-01-13|6.741666666666667           |686       |
|boston|2017-01-14|7.1                  

In [119]:
trip_data_group_by_date_pd_df = trip_data_group_by_date_df.toPandas()

# trip_data_group_by_date_pd_df[trip_data_group_by_date_pd_df['city'] == 'nyc']['trip_count_sma_7d'] = \
#     trip_data_group_by_date_pd_df[trip_data_group_by_date_pd_df['city'] == 'nyc']['trip_count'].rolling(window=7).mean()

# trip_data_group_by_date_pd_df['trip_count_sma_14d'] = trip_data_group_by_date_pd_df['trip_count'].rolling(window=14).mean()
display(trip_data_group_by_date_pd_df.head())
display(trip_data_group_by_date_pd_df.describe())

,city,trip_date,median_trip_duration_minutes,trip_count
0,boston,2017-01-01,9.216667,481
1,boston,2017-01-02,8.700000,801
2,boston,2017-01-03,8.633333,651
3,boston,2017-01-04,9.016667,1534
4,boston,2017-01-05,8.700000,1330


,median_trip_duration_minutes,trip_count
count,5017.000000,5017.000000
mean,11.049520,22880.181981
std,2.543224,28472.596177
min,6.033333,21.000000
25%,9.333333,4218.000000
50%,10.450000,7545.000000
75%,12.133333,37372.000000
max,28.416667,137709.000000


In [162]:
trip_data_group_by_date_pd_df = trip_data_group_by_date_df.toPandas()

In [163]:
trip_data_boston_pd_df = trip_data_group_by_date_pd_df[trip_data_group_by_date_pd_df['city']=='boston'].copy()
trip_data_boston_pd_df.reset_index(drop=True, inplace=True)
trip_data_boston_pd_df['trip_count_7d_ma'] = trip_data_boston_pd_df['trip_count'].rolling(7, 1).mean()
trip_data_boston_pd_df['trip_count_14d_ma'] = trip_data_boston_pd_df['trip_count'].rolling(14, 1).mean()
trip_data_boston_pd_df.head(10)

,city,trip_date,median_trip_duration_minutes,trip_count,trip_count_7d_ma,trip_count_14d_ma
0,boston,2017-01-01,9.216667,481,481.000000,481.000000
1,boston,2017-01-02,8.700000,801,641.000000,641.000000
2,boston,2017-01-03,8.633333,651,644.333333,644.333333
3,boston,2017-01-04,9.016667,1534,866.750000,866.750000
4,boston,2017-01-05,8.700000,1330,959.400000,959.400000
5,boston,2017-01-06,8.083333,835,938.666667,938.666667
6,boston,2017-01-07,7.175000,106,819.714286,819.714286
7,boston,2017-01-08,7.866667,111,766.857143,731.125000
8,boston,2017-01-09,7.091667,392,708.428571,693.444444
9,boston,2017-01-10,7.233333,469,682.428571,671.000000


In [164]:
trip_data_nyc_pd_df = trip_data_group_by_date_pd_df[trip_data_group_by_date_pd_df['city']=='nyc'].copy()
trip_data_nyc_pd_df.reset_index(drop=True, inplace=True)
trip_data_nyc_pd_df['trip_count_7d_ma'] = trip_data_nyc_pd_df['trip_count'].rolling(7, 1).mean()
trip_data_nyc_pd_df['trip_count_14d_ma'] = trip_data_nyc_pd_df['trip_count'].rolling(14, 1).mean()
trip_data_nyc_pd_df.head(10)

,city,trip_date,median_trip_duration_minutes,trip_count,trip_count_7d_ma,trip_count_14d_ma
0,nyc,2017-01-01,11.100000,16273,16273.000000,16273.000000
1,nyc,2017-01-02,7.450000,9061,12667.000000,12667.000000
2,nyc,2017-01-03,8.133333,14543,13292.333333,13292.333333
3,nyc,2017-01-04,8.966667,34669,18636.500000,18636.500000
4,nyc,2017-01-05,8.316667,28942,20697.600000,20697.600000
5,nyc,2017-01-06,8.366667,24599,21347.833333,21347.833333
6,nyc,2017-01-07,7.816667,4494,18940.142857,18940.142857
7,nyc,2017-01-08,8.166667,6522,17547.142857,17387.875000
8,nyc,2017-01-09,8.466667,16173,18563.142857,17252.888889
9,nyc,2017-01-10,8.700000,23633,19861.714286,17890.900000


In [165]:
trip_data_sf_pd_df = trip_data_group_by_date_pd_df[trip_data_group_by_date_pd_df['city']=='sf'].copy()
trip_data_sf_pd_df.reset_index(drop=True, inplace=True)
trip_data_sf_pd_df['trip_count_7d_ma'] = trip_data_sf_pd_df['trip_count'].rolling(7, 1).mean()
trip_data_sf_pd_df['trip_count_14d_ma'] = trip_data_sf_pd_df['trip_count'].rolling(14, 1).mean()
trip_data_sf_pd_df.head(10)

,city,trip_date,median_trip_duration_minutes,trip_count,trip_count_7d_ma,trip_count_14d_ma
0,sf,2017-06-28,9.458333,632,632.000000,632.000000
1,sf,2017-06-29,9.350000,1019,825.500000,825.500000
2,sf,2017-06-30,9.366667,1038,896.333333,896.333333
3,sf,2017-07-01,16.783333,475,791.000000,791.000000
4,sf,2017-07-02,18.416667,523,737.400000,737.400000
5,sf,2017-07-03,10.958333,600,714.500000,714.500000
6,sf,2017-07-04,17.866667,558,692.142857,692.142857
7,sf,2017-07-05,9.533333,1103,759.428571,743.500000
8,sf,2017-07-06,9.533333,1277,796.285714,802.777778
9,sf,2017-07-07,9.600000,1126,808.857143,835.100000


In [207]:
import pandas as pd
from sklearn import preprocessing

trip_data_by_city_df = [trip_data_boston_pd_df, trip_data_nyc_pd_df, trip_data_sf_pd_df]

for city_trip_data in trip_data_by_city_df:
    x = city_trip_data[['median_trip_duration_minutes', 'trip_count', 'trip_count_7d_ma', 'trip_count_14d_ma']].values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)

    new_columns = ['median_trip_duration_minutes_norm', 'trip_count_norm', 'trip_count_7d_ma_norm', 'trip_count_14d_ma_norm']
    scaled_df = pd.DataFrame(x_scaled, columns=new_columns)

    for column in new_columns:
        city_trip_data[column] = scaled_df[column]
    


trip_data_pd_df = pd.concat(trip_data_by_city_df)
trip_data_pd_df.reset_index(drop=True, inplace=True)
display(trip_data_pd_df.head(10))
display(trip_data_pd_df.tail(10))

trip_data_group_by_date_pd_df = trip_data_pd_df   
trip_data_group_by_date_pd_df.head()    


,city,trip_date,median_trip_duration_minutes,trip_count,trip_count_7d_ma,trip_count_14d_ma,median_trip_duration_minutes_norm,trip_count_norm,trip_count_7d_ma_norm,trip_count_14d_ma_norm
0,boston,2017-01-01,9.216667,481,481.000000,481.000000,0.142219,0.025360,0.016195,0.009256
1,boston,2017-01-02,8.700000,801,641.000000,641.000000,0.119136,0.043001,0.026679,0.019937
2,boston,2017-01-03,8.633333,651,644.333333,644.333333,0.116158,0.034732,0.026898,0.020159
3,boston,2017-01-04,9.016667,1534,866.750000,866.750000,0.133284,0.083411,0.041472,0.035008
4,boston,2017-01-05,8.700000,1330,959.400000,959.400000,0.119136,0.072165,0.047544,0.041193
5,boston,2017-01-06,8.083333,835,938.666667,938.666667,0.091586,0.044876,0.046185,0.039809
6,boston,2017-01-07,7.175000,106,819.714286,819.714286,0.051005,0.004686,0.038390,0.031868
7,boston,2017-01-08,7.866667,111,766.857143,731.125000,0.081906,0.004962,0.034927,0.025954
8,boston,2017-01-09,7.091667,392,708.428571,693.444444,0.047282,0.020453,0.031098,0.023438
9,boston,2017-01-10,7.233333,469,682.428571,671.000000,0.053611,0.024698,0.029394,0.021940


,city,trip_date,median_trip_duration_minutes,trip_count,trip_count_7d_ma,trip_count_14d_ma,median_trip_duration_minutes_norm,trip_count_norm,trip_count_7d_ma_norm,trip_count_14d_ma_norm
5007,sf,2021-09-21,10.233333,7140,7421.142857,7188.857143,0.241287,0.394715,0.449047,0.449577
5008,sf,2021-09-22,10.050000,6993,7430.285714,7194.357143,0.226542,0.386064,0.449652,0.449954
5009,sf,2021-09-23,9.983333,7203,7494.142857,7237.000000,0.221180,0.398423,0.453875,0.452878
5010,sf,2021-09-24,10.400000,7527,7474.571429,7276.357143,0.254692,0.417491,0.452581,0.455577
5011,sf,2021-09-25,11.350000,7470,7360.714286,7233.928571,0.331099,0.414136,0.445050,0.452667
5012,sf,2021-09-26,11.266667,6909,7120.000000,7234.214286,0.324397,0.381121,0.429129,0.452687
5013,sf,2021-09-27,9.633333,5378,6945.714286,7162.428571,0.193029,0.291019,0.417601,0.447765
5014,sf,2021-09-28,10.133333,7270,6964.285714,7192.714286,0.233244,0.402366,0.418830,0.449842
5015,sf,2021-09-29,10.133333,7297,7007.714286,7219.000000,0.233244,0.403955,0.421702,0.451644
5016,sf,2021-09-30,10.483333,7929,7111.428571,7302.785714,0.261394,0.441149,0.428562,0.457389


,city,trip_date,median_trip_duration_minutes,trip_count,trip_count_7d_ma,trip_count_14d_ma,median_trip_duration_minutes_norm,trip_count_norm,trip_count_7d_ma_norm,trip_count_14d_ma_norm
0,boston,2017-01-01,9.216667,481,481.000000,481.000000,0.142219,0.025360,0.016195,0.009256
1,boston,2017-01-02,8.700000,801,641.000000,641.000000,0.119136,0.043001,0.026679,0.019937
2,boston,2017-01-03,8.633333,651,644.333333,644.333333,0.116158,0.034732,0.026898,0.020159
3,boston,2017-01-04,9.016667,1534,866.750000,866.750000,0.133284,0.083411,0.041472,0.035008
4,boston,2017-01-05,8.700000,1330,959.400000,959.400000,0.119136,0.072165,0.047544,0.041193


In [208]:

# trip_data_group_by_date_pd_df = trip_data_group_by_date_pd_df.sort_values(by=['city', 'trip_date'])
# trip_data_group_by_date_pd_df.head()

# boston_trip_data_group_by_date_pd_df = trip_data_group_by_date_pd_df[trip_data_group_by_date_pd_df['city'] == 'boston']

# boston_trip_data_group_by_date_pd_df['rolling'] = boston_trip_data_group_by_date_pd_df['trip_count'].rolling(7, 1).mean()
# boston_trip_data_group_by_date_pd_df.head(10)

# trip_data_group_by_date_pd_df['trip_count_7d_ma'] = \
#     trip_data_group_by_date_pd_df.groupby(['trip_date', 'city'])['trip_count'].rolling(7, 1).mean() \
#         .reset_index(drop=True)
# #         .transform(lambda x: x.rolling(7, 1).mean())


# blah_df = trip_data_group_by_date_pd_df.groupby(['trip_date', 'city'], as_index=False)['trip_count'].rolling(3).mean().reset_index(level=0,drop=True)

# display(blah_df)

# trip_data_group_by_date_pd_df['moving'] = trip_data_group_by_date_pd_df.groupby(['city', 'trip_date'])['trip_count'].transform(lambda x: x.rolling(7).mean())

# trip_data_group_by_date_pd_df = trip_data_group_by_date_pd_df[['trip_date', 'city', 'trip_count']]

# trip_data_group_by_date_pd_df['moving'] = trip_data_group_by_date_pd_df.groupby(['trip_date', 'city']).rolling(7)['trip_count'].mean().reset_index(drop=True)

# df.loc[:, 'value_sma_10'] = \
#     df.groupby('city')[['city', 'trip_date']].rolling(window=8, min_periods=1, on='trip_date').mean().reset_index(level='city')['trip_count']


# trip_data_group_by_date_pd_df = (trip_data_group_by_date_pd_df.assign(moving_avg = lambda x:  x.groupby(['trip_date', 'city'], as_index=False)['trip_count'] \
#                                     .transform(lambda x: x.rolling(7)).mean()))
                                      
#                                     .reset_index(drop=True)))


# trip_data_group_by_date_pd_df[trip_data_group_by_date_pd_df['city'] == 'boston'].head(10)

# trip_data_group_by_date_pd_df.head()

# df['moving'] = df.groupby(['col_1', 'col_2', 'col_3']).rolling(10)['value'].mean().droplevel(level=[0,1,2])



In [209]:
trip_data_group_by_date_pd_df[trip_data_group_by_date_pd_df['city'] == 'boston'].head(10)

,city,trip_date,median_trip_duration_minutes,trip_count,trip_count_7d_ma,trip_count_14d_ma,median_trip_duration_minutes_norm,trip_count_norm,trip_count_7d_ma_norm,trip_count_14d_ma_norm
0,boston,2017-01-01,9.216667,481,481.000000,481.000000,0.142219,0.025360,0.016195,0.009256
1,boston,2017-01-02,8.700000,801,641.000000,641.000000,0.119136,0.043001,0.026679,0.019937
2,boston,2017-01-03,8.633333,651,644.333333,644.333333,0.116158,0.034732,0.026898,0.020159
3,boston,2017-01-04,9.016667,1534,866.750000,866.750000,0.133284,0.083411,0.041472,0.035008
4,boston,2017-01-05,8.700000,1330,959.400000,959.400000,0.119136,0.072165,0.047544,0.041193
5,boston,2017-01-06,8.083333,835,938.666667,938.666667,0.091586,0.044876,0.046185,0.039809
6,boston,2017-01-07,7.175000,106,819.714286,819.714286,0.051005,0.004686,0.038390,0.031868
7,boston,2017-01-08,7.866667,111,766.857143,731.125000,0.081906,0.004962,0.034927,0.025954
8,boston,2017-01-09,7.091667,392,708.428571,693.444444,0.047282,0.020453,0.031098,0.023438
9,boston,2017-01-10,7.233333,469,682.428571,671.000000,0.053611,0.024698,0.029394,0.021940


In [210]:
trip_data_group_by_date_pd_df.to_csv('2017_present_trip_data.csv')

In [211]:
fig = px.line(trip_data_group_by_date_pd_df, x="trip_date", y="median_trip_duration_minutes", color='city')
fig.show()

In [212]:
fig = px.line(trip_data_group_by_date_pd_df, x="trip_date", y="median_trip_duration_minutes_norm", color='city')
fig.show()

In [213]:
fig = px.line(trip_data_group_by_date_pd_df, x="trip_date", y="trip_count", color='city')
fig.show()

In [214]:
fig = px.line(trip_data_group_by_date_pd_df, x="trip_date", y="trip_count_norm", color='city')
fig.show()

In [215]:
fig = px.line(trip_data_group_by_date_pd_df, x="trip_date", y="trip_count_7d_ma", color='city')
fig.show()

In [216]:
fig = px.line(trip_data_group_by_date_pd_df, x="trip_date", y="trip_count_7d_ma_norm", color='city')
fig.show()

In [217]:
fig = px.line(trip_data_group_by_date_pd_df, x="trip_date", y="trip_count_14d_ma", color='city')
fig.show()

In [218]:
fig = px.line(trip_data_group_by_date_pd_df, x="trip_date", y="trip_count_14d_ma_norm", color='city')
fig.show()